# Training Model v2.0.2 - F2-Score Optimized (Binary Crossentropy Loss)

**Objective:** Maximize F2-Score while maintaining balanced Precision

**Key Features:**
- Loss function: `binary_crossentropy` (proper probabilistic loss)
- Optimization target: `val_f2_score` (primary) + `val_precision` (secondary constraint)
- Fixed decision threshold: 0.5 for all metrics
- Prevents pathological solutions (high recall, low precision)

**Artifacts Saved:**
- `Models/model_v2.0.2.h5` - Final trained model
- `Models/scaler_v2.0.2.pkl` - Preprocessing scaler

**NOT Saved:**
- Hyperparameter trial logs
- Training history
- Checkpoints


In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

# Load dataset
df = pd.read_csv("../dataset_rio_2km_week_encoded.csv")

print(f"✓ Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"✓ Target distribution:")
print(f"  - No flood: {(df['seInunda'] == 0).sum()} ({100*(df['seInunda'] == 0).sum()/len(df):.1f}%)")
print(f"  - Flood: {(df['seInunda'] == 1).sum()} ({100*(df['seInunda'] == 1).sum()/len(df):.1f}%)")


✓ Dataset loaded: 5531 rows, 29 columns
✓ Target distribution:
  - No flood: 2805 (50.7%)
  - Flood: 2726 (49.3%)


In [9]:
# ============================================================================
# DATA PREPARATION
# ============================================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Define features and target
FEATURES = [
    'hayRioCercano', 'elevacion_m', 'pendiente_grados', 
    'temperature_2m', 'total_evaporation_sum', 
    'volumetric_soil_water_layer_1', 'precipitation_week',
    # Soil types (One-Hot Encoded)
    'tipoDeSuelo_Alfisoles', 'tipoDeSuelo_Aridisoles',
    'tipoDeSuelo_Complejo indiferenci', 'tipoDeSuelo_Entisoles',
    'tipoDeSuelo_Esteros', 'tipoDeSuelo_Inceptisoles',
    'tipoDeSuelo_Lagunas', 'tipoDeSuelo_Medano', 'tipoDeSuelo_Miscelaneas',
    'tipoDeSuelo_Molisoles', 'tipoDeSuelo_Rio', 'tipoDeSuelo_Roca',
    'tipoDeSuelo_Salinas', 'tipoDeSuelo_Ultisoles', 'tipoDeSuelo_Vertisoles',
    # Seasons (One-Hot Encoded)
    'estacion_invierno', 'estacion_otoño', 'estacion_primavera', 'estacion_verano'
]
TARGET = 'seInunda'

# Extract features and target
X = df[FEATURES].values.astype(np.float32)
y = df[TARGET].values.astype(np.int32)

print(f"✓ Features shape: {X.shape} ({len(FEATURES)} features)")
print(f"✓ Target shape: {y.shape}")

# Train/test split with stratification (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✓ Train set: {X_train.shape[0]} samples")
print(f"✓ Test set: {X_test.shape[0]} samples")

# Standardize features (fit on train, transform both)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Data preprocessed and ready for training")


✓ Features shape: (5531, 26) (26 features)
✓ Target shape: (5531,)

✓ Train set: 4424 samples
✓ Test set: 1107 samples

✓ Data preprocessed and ready for training


In [10]:
# ============================================================================
# CUSTOM F2-SCORE METRIC (METRIC ONLY, NOT LOSS)
# ============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

class F2Score(keras.metrics.Metric):
    """
    Custom F2-Score metric for Keras.
    
    F2 = (1 + β²) × (precision × recall) / (β² × precision + recall), where β=2
    
    This gives more weight to recall (80%) than precision (20%).
    
    IMPORTANT: This is a METRIC only, not used as a loss function.
    The model uses binary_crossentropy as loss.
    """
    def __init__(self, name='f2_score', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision_metric = keras.metrics.Precision(thresholds=threshold)
        self.recall_metric = keras.metrics.Recall(thresholds=threshold)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision_metric.update_state(y_true, y_pred, sample_weight)
        self.recall_metric.update_state(y_true, y_pred, sample_weight)
    
    def result(self):
        precision = self.precision_metric.result()
        recall = self.recall_metric.result()
        
        # F2 = 5 * (precision * recall) / (4 * precision + recall)
        numerator = 5.0 * precision * recall
        denominator = 4.0 * precision + recall
        
        return tf.where(
            tf.equal(denominator, 0),
            0.0,
            numerator / denominator
        )
    
    def reset_state(self):
        self.precision_metric.reset_state()
        self.recall_metric.reset_state()


print("✓ Custom F2-Score metric defined (for evaluation only)")
print("✓ Loss function will be: binary_crossentropy")
print("✓ All metrics use fixed threshold: 0.5")


✓ Custom F2-Score metric defined (for evaluation only)
✓ Loss function will be: binary_crossentropy
✓ All metrics use fixed threshold: 0.5


In [11]:
# ============================================================================
# HYPERPARAMETER SEARCH WITH KERAS TUNER
# ============================================================================

def build_model(hp):
    """
    Build a model with tunable hyperparameters.
    
    IMPORTANT CHANGES:
    - Loss: binary_crossentropy (NOT F2 loss)
    - Metrics: All use threshold=0.5
    - Tuner optimizes: val_f2_score (primary) + val_precision (constraint)
    
    Search space:
    - Number of layers: 2-5
    - Units per layer: 32-256
    - Dropout rates: 0.1-0.5
    - Learning rate: 1e-4 to 1e-2
    - Activation functions: relu, elu, swish
    """
    model = keras.Sequential()
    
    # Input layer
    input_dim = X_train_scaled.shape[1]
    model.add(layers.InputLayer(input_shape=(input_dim,)))
    
    # Tunable number of hidden layers
    n_layers = hp.Int('n_layers', min_value=2, max_value=5, step=1)
    
    for i in range(n_layers):
        # Tunable units per layer
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32)
        
        # Tunable activation
        activation = hp.Choice(f'activation_{i}', values=['relu', 'elu', 'swish'])
        
        # Add dense layer with L2 regularization
        model.add(layers.Dense(
            units, 
            activation=activation,
            kernel_regularizer=keras.regularizers.l2(0.001)
        ))
        
        # Tunable dropout rate
        dropout = hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)
        model.add(layers.Dropout(dropout))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Tunable learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-4, 5e-4, 1e-3, 5e-3, 1e-2])
    
    # Compile with BINARY CROSSENTROPY loss (proper probabilistic loss)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',  # Standard loss for binary classification
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision', thresholds=0.5),  # Fixed threshold
            keras.metrics.Recall(name='recall', thresholds=0.5),        # Fixed threshold
            F2Score(name='f2_score', threshold=0.5)                     # Fixed threshold
        ]
    )
    
    return model


# Configure the tuner with dual objectives
print("\n" + "="*70)
print("HYPERPARAMETER SEARCH CONFIGURATION")
print("="*70)

# Primary objective: Maximize F2-Score
# Secondary constraint: Maintain reasonable precision
tuner = kt.RandomSearch(
    build_model,
    objective=[
        kt.Objective('val_f2_score', direction='max'),      # Primary: maximize F2
        kt.Objective('val_precision', direction='max')      # Secondary: maintain precision
    ],
    max_trials=30,
    executions_per_trial=2,
    directory='../hyperparameter_tuning',
    project_name='flood_prediction_crossentropy_v2.0.2',
    overwrite=False
)

print("\n✓ Tuner configured:")
print(f"  - Loss: binary_crossentropy (NOT F2 loss)")
print(f"  - Primary objective: Maximize val_f2_score")
print(f"  - Secondary objective: Maximize val_precision")
print(f"  - This prevents models with recall≈1.0 and precision≈0.0")
print(f"  - Max trials: 30")
print(f"  - Executions per trial: 2")
print(f"\n✓ Search space:")
print(f"  - Layers: 2-5")
print(f"  - Units per layer: 32-256 (step 32)")
print(f"  - Dropout: 0.1-0.5 (step 0.1)")
print(f"  - Learning rate: [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]")
print(f"  - Activation: [relu, elu, swish]")
print(f"\n✓ All metrics use fixed threshold: 0.5")



HYPERPARAMETER SEARCH CONFIGURATION

✓ Tuner configured:
  - Loss: binary_crossentropy (NOT F2 loss)
  - Primary objective: Maximize val_f2_score
  - Secondary objective: Maximize val_precision
  - This prevents models with recall≈1.0 and precision≈0.0
  - Max trials: 30
  - Executions per trial: 2

✓ Search space:
  - Layers: 2-5
  - Units per layer: 32-256 (step 32)
  - Dropout: 0.1-0.5 (step 0.1)
  - Learning rate: [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
  - Activation: [relu, elu, swish]

✓ All metrics use fixed threshold: 0.5


C:\Users\COMPU\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [12]:
# ============================================================================
# HYPERPARAMETER SEARCH EXECUTION
# ============================================================================

print("\n" + "="*70)
print("SEARCHING FOR BEST HYPERPARAMETERS")
print("="*70)
print("⚠️  This may take several minutes to hours depending on hardware...")
print()

# Configure early stopping for each trial
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss (binary crossentropy)
    mode='min',
    patience=10,
    restore_best_weights=True,
    verbose=0
)

# Run the hyperparameter search
tuner.search(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=0
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n" + "="*70)
print("🏆 BEST HYPERPARAMETERS FOUND")
print("="*70)
print(f"\nNumber of layers: {best_hps.get('n_layers')}")
for i in range(best_hps.get('n_layers')):
    print(f"  Layer {i+1}:")
    print(f"    - Units: {best_hps.get(f'units_{i}')}")
    print(f"    - Activation: {best_hps.get(f'activation_{i}')}")
    print(f"    - Dropout: {best_hps.get(f'dropout_{i}')}")
print(f"\nLearning rate: {best_hps.get('learning_rate')}")

# Get best models summary
print("\n" + "="*70)
print("TOP 3 MODELS BY F2-SCORE")
print("="*70)
for idx, model_trial in enumerate(tuner.oracle.get_best_trials(num_trials=3), 1):
    f2_score = model_trial.metrics.get_best_value('val_f2_score')
    precision = model_trial.metrics.get_best_value('val_precision')
    recall = model_trial.metrics.get_best_value('val_recall')
    print(f"\nRank {idx}:")
    print(f"  F2-Score: {f2_score:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")



SEARCHING FOR BEST HYPERPARAMETERS
⚠️  This may take several minutes to hours depending on hardware...






🏆 BEST HYPERPARAMETERS FOUND

Number of layers: 3
  Layer 1:
    - Units: 192
    - Activation: relu
    - Dropout: 0.2
  Layer 2:
    - Units: 128
    - Activation: relu
    - Dropout: 0.4
  Layer 3:
    - Units: 224
    - Activation: swish
    - Dropout: 0.1

Learning rate: 0.005

TOP 3 MODELS BY F2-SCORE

Rank 1:
  F2-Score: 0.8396
  Precision: 0.7023
  Recall: 0.8834

Rank 2:
  F2-Score: 0.8207
  Precision: 0.7184
  Recall: 0.8510

Rank 3:
  F2-Score: 0.8012
  Precision: 0.7307
  Recall: 0.8210


In [13]:
# ============================================================================
# TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS
# ============================================================================

print("\n" + "="*70)
print("TRAINING FINAL MODEL")
print("="*70)

# Build the best model
model = tuner.hypermodel.build(best_hps)

print("\n📊 Model Architecture (with best hyperparameters):")
model.summary()

# Configure callbacks for final training
early_stopping_final = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor binary crossentropy loss
    mode='min',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# Train the final model
print("\n🚀 Training...")
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=32,
    callbacks=[early_stopping_final],
    verbose=1
)

print("\n✓ Training completed")
print(f"✓ Best validation loss: {min(history.history['val_loss']):.4f}")
print(f"✓ Best validation F2-Score: {max(history.history['val_f2_score']):.4f}")
print(f"✓ Final validation precision: {history.history['val_precision'][-1]:.4f}")
print(f"✓ Final validation recall: {history.history['val_recall'][-1]:.4f}")



TRAINING FINAL MODEL

📊 Model Architecture (with best hyperparameters):


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 192)            │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 224)            │        28,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,009 (230.50 KB)

 Trainable params: 59,009 (230.50 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Training...
Epoch 1/150
111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6880 - f2_score: 0.6886 - loss: 0.7859 - precision: 0.6817 - recall: 0.6903 - val_accuracy: 0.7288 - val_f2_score: 0.8118 - val_loss: 0.6511 - val_precision: 0.6764 - val_recall: 0.8545
Epoch 2/150
111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6880 - f2_score: 0.6886 - loss: 0.7859 - precision: 0.6817 - recall: 0.6903 - val_accuracy: 0.7288 - val_f2_score: 0.8118 - val_loss: 0.6511 - val_precision: 0.6764 - val_recall: 0.8545
Epoch 2/150
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7177 - f2_score: 0.7460 - loss: 0.6378 - precision: 0.6962 - recall: 0.7596 - val_accuracy: 0.7299 - val_f2_score: 0.7865 - val_loss: 0.5878 - val_precision: 0.6895 - val_recall: 0.8152
Epoch 3/150
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7177 - f2_score: 0.7460 - loss: 0.6378 - precision: 0.6962 - recall: 0.7596 - val_accuracy: 0.7299 - val_f2_score: 0.7865 - val_loss: 0.5878 - val_precision: 0

In [14]:
# ============================================================================
# EVALUATE MODEL ON TEST SET
# ============================================================================

print("\n" + "="*70)
print("EVALUATING ON TEST SET")
print("="*70)

# Evaluate with automatic metrics (threshold=0.5 fixed)
test_loss, test_acc, test_prec, test_rec, test_f2 = model.evaluate(
    X_test_scaled, y_test,
    verbose=0
)

print("\n📈 Test Metrics (threshold=0.5):")
print(f"  • Loss (binary crossentropy): {test_loss:.4f}")
print(f"  • Accuracy: {test_acc:.4f}")
print(f"  • Precision: {test_prec:.4f}")
print(f"  • Recall: {test_rec:.4f}")
print(f"  • F2-Score: {test_f2:.4f}")

# Generate predictions for additional validation
y_pred_proba = model.predict(X_test_scaled, verbose=0).flatten()
y_pred = (y_pred_proba >= 0.5).astype(int)

# Check for pathological solutions
positive_rate = y_pred.mean()
print("\n🔍 Pathological Solution Check:")
print(f"  • Positive prediction rate: {positive_rate:.2%}")
print(f"  • Test precision: {test_prec:.4f}")

if positive_rate > 0.80:
    print("  ⚠️ WARNING: Model predicts positive >80% of the time")
if test_prec < 0.30:
    print("  ⚠️ WARNING: Precision is very low (<0.30)")
    
if positive_rate <= 0.80 and test_prec >= 0.30:
    print("  ✓ Model appears balanced (precision ≥ 0.30, positive rate ≤ 80%)")

# Show confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("\n📊 Confusion Matrix:")
print(f"  TN={cm[0,0]:5d}  FP={cm[0,1]:5d}")
print(f"  FN={cm[1,0]:5d}  TP={cm[1,1]:5d}")



EVALUATING ON TEST SET

📈 Test Metrics (threshold=0.5):
📈 Test Metrics (threshold=0.5):
  • Loss (binary crossentropy): 0.5450
  • Accuracy: 0.7425
  • Precision: 0.7150
  • Recall: 0.7949
  • F2-Score: 0.7775

  • Loss (binary crossentropy): 0.5450
  • Accuracy: 0.7425
  • Precision: 0.7150
  • Recall: 0.7949
  • F2-Score: 0.7775

🔍 Pathological Solution Check:
  • Positive prediction rate: 54.83%
  • Test precision: 0.7150
  ✓ Model appears balanced (precision ≥ 0.30, positive rate ≤ 80%)

📊 Confusion Matrix:
  TN=  388  FP=  173
  FN=  112  TP=  434

🔍 Pathological Solution Check:
  • Positive prediction rate: 54.83%
  • Test precision: 0.7150
  ✓ Model appears balanced (precision ≥ 0.30, positive rate ≤ 80%)

📊 Confusion Matrix:
  TN=  388  FP=  173
  FN=  112  TP=  434


In [15]:
# ============================================================================
# SAVE ARTIFACTS (Model, Scaler, Hyperparameters, Metadata)
# ============================================================================

def save_artifacts(model, scaler, best_hps, version='2.0.2'):
    """
    Save only essential artifacts for model deployment.
    
    Saves:
    - model_v{version}.h5: Trained Keras model
    - scaler_v{version}.pkl: StandardScaler for preprocessing
    - hyperparameters_v{version}.json: Best hyperparameters found
    - metadata_v{version}.txt: Human-readable summary
    """
    import os
    import joblib
    from datetime import datetime
    
    # Create Models directory if it doesn't exist
    models_dir = 'Models'
    os.makedirs(models_dir, exist_ok=True)
    
    # Define file paths with related names
    model_path = os.path.join(models_dir, f'model_v{version}.h5')
    scaler_path = os.path.join(models_dir, f'scaler_v{version}.pkl')
    hps_path = os.path.join(models_dir, f'hyperparameters_v{version}.json')
    metadata_path = os.path.join(models_dir, f'metadata_v{version}.txt')
    
    # Save model
    model.save(model_path)
    print(f"\n✓ Model saved: {model_path}")
    
    # Save scaler
    joblib.dump(scaler, scaler_path)
    print(f"✓ Scaler saved: {scaler_path}")
    
    # Save hyperparameters
    hps_dict = best_hps.values.copy()
    with open(hps_path, 'w') as f:
        json.dump(hps_dict, f, indent=2)
    print(f"✓ Hyperparameters saved: {hps_path}")
    
    # Save metadata
    with open(metadata_path, 'w') as f:
        f.write(f"Model Version: {version}\n")
        f.write(f"Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Number of features: {len(FEATURES)}\n")
        f.write(f"\nTest Performance (threshold=0.5):\n")
        f.write(f"  F2-Score: {test_f2:.4f}\n")
        f.write(f"  Precision: {test_prec:.4f}\n")
        f.write(f"  Recall: {test_rec:.4f}\n")
        f.write(f"  Accuracy: {test_acc:.4f}\n")
        f.write(f"\nOptimization Strategy:\n")
        f.write(f"  Loss Function: binary_crossentropy\n")
        f.write(f"  Primary Objective: val_f2_score (maximize)\n")
        f.write(f"  Secondary Objective: val_precision (maximize)\n")
        f.write(f"  Threshold: 0.5 (fixed)\n")
        f.write(f"\nHyperparameters:\n")
        for key, value in hps_dict.items():
            f.write(f"  {key}: {value}\n")
        f.write(f"\nFeatures ({len(FEATURES)}):\n")
        for i, feat in enumerate(FEATURES, 1):
            f.write(f"  {i:2d}. {feat}\n")
    print(f"✓ Metadata saved: {metadata_path}")
    
    print(f"\n✅ All artifacts saved successfully to {models_dir}/")
    return model_path, scaler_path, hps_path, metadata_path

# Save everything
print("\n" + "="*70)
print("SAVING ARTIFACTS")
print("="*70)

model_path, scaler_path, hps_path, metadata_path = save_artifacts(
    model=model,
    scaler=scaler,
    best_hps=best_hps,
    version='2.0.2'
)



SAVING ARTIFACTS

✓ Model saved: Models\model_v2.0.2.h5
✓ Scaler saved: Models\scaler_v2.0.2.pkl


NameError: name 'json' is not defined

In [ ]:
# ============================================================================
# USAGE EXAMPLE: How to load and use the saved model
# ============================================================================

print("\n" + "="*70)
print("USAGE EXAMPLE")
print("="*70)

import json
import joblib
from tensorflow import keras

# Load saved artifacts
print("\n📦 Loading saved artifacts...")
loaded_model = keras.models.load_model(
    model_path,
    custom_objects={'F2Score': F2Score}  # IMPORTANT: Register custom metric
)
loaded_scaler = joblib.load(scaler_path)

with open(hps_path, 'r') as f:
    loaded_hps = json.load(f)

print("✓ Model loaded successfully")
print("✓ Scaler loaded successfully")
print("✓ Hyperparameters loaded successfully")

# Example prediction on test set (first 5 samples)
print("\n🔮 Example predictions (first 5 test samples):")
print("-" * 70)

sample_X = X_test[:5]
sample_y_true = y_test[:5]

# Preprocess and predict
sample_X_scaled = loaded_scaler.transform(sample_X)
sample_y_pred_proba = loaded_model.predict(sample_X_scaled, verbose=0).flatten()
sample_y_pred = (sample_y_pred_proba >= 0.5).astype(int)  # Fixed threshold 0.5

# Display results
for i in range(5):
    true_label = "Flood" if sample_y_true[i] == 1 else "No Flood"
    pred_label = "Flood" if sample_y_pred[i] == 1 else "No Flood"
    prob = sample_y_pred_proba[i]
    
    match = "✓" if sample_y_true[i] == sample_y_pred[i] else "✗"
    print(f"Sample {i+1}: True={true_label:8s} | Pred={pred_label:8s} | Prob={prob:.3f} | {match}")

print("\n" + "="*70)
print("✅ WORKFLOW COMPLETE")
print("="*70)
print("\nKey Points:")
print("  • Loss function: binary_crossentropy (proper probabilistic training)")
print("  • F2-score: metric only (not loss)")
print("  • Optimization: dual objectives (F2-score + precision)")
print("  • Threshold: 0.5 (fixed for all metrics)")
print("  • Artifacts saved to: Models/")
print("\nThe model is ready for deployment!")



PRODUCTION USAGE EXAMPLE


✓ Model and scaler loaded successfully

Example predictions:
Sample   Probability     Prediction      Actual    
--------------------------------------------------
1        0.9947 ( 99.5%)  Flood           No Flood   ✗
2        0.9986 ( 99.9%)  Flood           No Flood   ✗
3        1.0000 (100.0%)  Flood           Flood      ✓
4        0.9962 ( 99.6%)  Flood           No Flood   ✗
5        1.0000 (100.0%)  Flood           Flood      ✓

✅ Successfully demonstrated how to load and use the model

Example predictions:
Sample   Probability     Prediction      Actual    
--------------------------------------------------
1        0.9947 ( 99.5%)  Flood           No Flood   ✗
2        0.9986 ( 99.9%)  Flood           No Flood   ✗
3        1.0000 (100.0%)  Flood           Flood      ✓
4        0.9962 ( 99.6%)  Flood           No Flood   ✗
5        1.0000 (100.0%)  Flood           Flood      ✓

✅ Successfully demonstrated how to load and use the model
